# Visualizing fire scares through false color 
### Author: Isabella Segarra 

### Objective:

### About the data:


In [23]:
# Environment set-up 
import pandas as pd # for data wrangling
import numpy as np  # for data wrangling
import geopandas as gpd
import os # for data import
import matplotlib.pyplot as plt  # for visualizations 
import xarray as xr # for Netcdf4 data 

### Data Import

In [4]:
# Import fire perimeter data
fp_eaton = os.path.join('data', 'Eaton_Perimeter_20250121')
eaton_perim = gpd.read_file(fp_eaton)

fp_palisades = os.path.join('data','Palisades_Perimeter_20250121')
palisades_perim = gpd.read_file(fp_palisades)

In [41]:
landsat = xr.open_dataset('data/landsat8-2025-02-23-palisades-eaton.nc', engine = 'scipy')

TypeError: Error: /Users/isabellasegarra/Documents/MEDS/EDS-220/eds220-hwk4/data/landsat8-2025-02-23-palisades-eaton.nc is not a valid NetCDF 3 file
            If this is a NetCDF4 file, you may need to install the
            netcdf4 library, e.g.,

            $ pip install netcdf4
            

In [ ]:
# Import Landsat data
landsat = xr.open_dataset('data/landsat8-2025-02-23-palisades-eaton.nc', engine = 'netcdf4')

ValueError: unrecognized engine netcdf4 must be one of your download engines: ['scipy', 'rasterio', 'store']To install additional dependencies, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

## Fire perimeter data exploration 
In this section I will do some preliminary exploration of the fire data. Since this is geospatial data, I will do the following:
- check the *CRS* of each dataset 
- check if the data is projected 
- check if the data is multipolygon or polygon 

In [ ]:
# Eaton fire perimeter

print('CRS:', eaton_perim.crs)
print('Is geographic?:', eaton_perim.crs.is_geographic)
print('Is projected?:', eaton_perim.crs.is_projected)
print('Is geometry polygon?:', eaton_perim.geom_type)

# Palisades fire perimeter
print('CRS:', palisades_perim.crs)
print('Is geographic?:', palisades_perim.crs.is_geographic)
print('Is projected?:', palisades_perim.crs.is_projected)
print('Is geometry polygon?:', palisades_perim.geom_type)

# Assert that they match 
eaton_perim.crs == palisades_perim.crs

**Exploration Findings**

From my preliminary exploration, I see that both datasets have matching coordinate refrene systems, are both projected, and all have polygon geometries. 